## Loading the data

In [ ]:
from read_data import VALID_TYPE_DATA, VALID_TASK_TYPES, read_data, get_dataset_name

# Use it here

## Preprocessing

### Downsampling

In [ ]:
from downsample import downsample

# Do the downsampling
# TODO: do we need this for X_train, y_train, X_test, y_test? Or just for train_data as one big array? If so, the method needs to be changed...

### Normalization

In [ ]:
from normalization import normalize

# Normalize the data

## Training

### Hyperparameters

In [ ]:
EPOCHS = []
# TODO: which hyperparameters are we going to grid search?

### Model definition

### Trainingloop

## Testing